In [ ]:
import pandas as pd
from scipy.stats import binomtest
import matplotlib.pyplot as plt

def transformer(div):
    d = []
    for item in div['div'][0]:
        if 'time' in item:
            d.append({
                'time': item['time'],
                'draw1': item['div'][1],
                'draw2': item['div'][2], 
                'draw3': item['div'][3], 
                'draw4': item['div'][4], 
                'draw5': item['div'][5],
                'draw6': item['div'][6],
                'draw7': item['div'][7],
            })
    return d

def convertColumnsInOneColumn(dataFrame):
    array = []
    for column in [str(x) for x in list(range(1,7))]:
        array.append(dataFrame["draw" + column])
    return array

def forAllFiles(fromYear, toYear):
    tmp = []
    for year in [str(x) for x in list(range(fromYear, toYear))]:
        div = pd.read_json("json/lotto" + year + ".json")
        transformed = pd.DataFrame(transformer(div))
        tmp.append(transformed)
    return tmp

summary = pd.concat(forAllFiles(1955, 2022))
overall = pd.concat(convertColumnsInOneColumn(summary)).value_counts()
newTable = pd.DataFrame({
    'Number': overall.index,
    'Count': overall.values
})

sumOverall = sum(newTable['Count'])
test = lambda x: binomtest(x, sumOverall, 1/49).pvalue
newTable['Test'] = newTable['Count'].apply(test)
newTable

#plt.figure(figsize=(9,6))
#plt.bar(newTable['Number'], newTable['Test'])


In [ ]:
from scipy.stats import binom
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt

numberOfTrials = 35
probOfSuccess = 0.7

success = {'success': list(range(0, numberOfTrials + 1))}
binomTable = pd.DataFrame(success)
prob = lambda x: binom.pmf(x, numberOfTrials, probOfSuccess)
binomTable['prob'] = binomTable['success'].apply(prob)

mu = numberOfTrials * probOfSuccess
sigma = np.sqrt(numberOfTrials * probOfSuccess * (1 - probOfSuccess))

ticks = {'ticks': np.arange(0, numberOfTrials, 0.1)}
normTable = pd.DataFrame(ticks)
prob = lambda x: norm.pdf(x, mu, sigma)
normTable['prob'] = normTable['ticks'].apply(prob)

plt.figure(figsize=(9,6))
plt.bar(binomTable['success'], binomTable['prob'])
plt.plot(normTable['ticks'], normTable['prob'])